In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import random
from PIL import Image
from numba import cuda

tf.keras.backend.clear_session()

In [2]:
data_dir = '../data/horses-vs-humans/'
train_dir = '../data/horses-vs-humans/train/'
valid_dir = '../data/horses-vs-humans/validation/'
train_horses_dir = '../data/horses-vs-humans/train/horses/'
train_humans_dir = '../data/horses-vs-humans/train/humans/'
valid_horses_dir = '../data/horses-vs-humans/validation/horses/'
valid_humans_dir = '../data/horses-vs-humans/validation/humans/'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_datagen = ImageDataGenerator(rescale= 1 / 255.0,
                                  vertical_flip = True,
                                  rotation_range=40,
                                  shear_range = 0.2,width_shift_range=0.2,zoom_range=0.2,fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale = 1 / 255.0)

In [5]:
batch_size = 16
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150),
                                                   batch_size=batch_size,
                                                   class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                   target_size=(150,150),
                                                   batch_size=batch_size,
                                                   class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),   #148x148
    tf.keras.layers.MaxPooling2D(2,2),                                            #74x74
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),                           #72x72
    tf.keras.layers.MaxPooling2D(2,2),                                            #36x36
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),                           #34x34
    tf.keras.layers.MaxPooling2D(2,2),                                            #17x17
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),                           #15x15
    tf.keras.layers.MaxPooling2D(2,2),                                            #7x7
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [7]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        3

In [8]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['acc'])

In [9]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        if logs.get('acc')>0.99:
            print("Reached 99% accuracy so cancelling training.")
            self.model.stop_training = True

In [10]:
callbacks = MyCallback()

In [11]:
history = model.fit_generator(train_generator,
                              validation_data=valid_generator,
                              steps_per_epoch=1027/batch_size,
                              epochs = 20,
                              validation_steps=256/batch_size,
                              shuffle=True,
                              verbose=1,
                             callbacks=[callbacks])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
65/64 [==============================] - 13s 203ms/step - loss: 0.6965 - acc: 0.5180 - val_loss: 0.6337 - val_acc: 0.6250
Epoch 2/20
65/64 [==============================] - 13s 202ms/step - loss: 0.5986 - acc: 0.6981 - val_loss: 0.8581 - val_acc: 0.6797
Epoch 3/20
65/64 [==============================] - 14s 213ms/step - loss: 0.4419 - acc: 0.7877 - val_loss: 0.8194 - val_acc: 0.7734
Epoch 4/20
65/64 [==============================] - 14s 222ms/step - loss: 0.3710 - acc: 0.8257 - val_loss: 1.3828 - val_acc: 0.7070
Epoch 5/20
65/64 [==============================] - 14s 210ms/step - loss: 0.2779 - acc: 0.8744 - val_loss: 1.7000 - val_acc: 0.6953
Epoch 6/20
65/64 [==============================] - 15s 225ms/step - loss: 0.2325 - acc: 0.9133 - val_loss: 3.0747 - val_acc: 0.7109
Epoch 7/20
65/64 [==============================] - 18s 277ms/step - loss: 0.2447 - acc: 0.9114 - val_loss: 1.5648 - val_acc: 

In [20]:
from keras.preprocessing import image

path = '../data/horses-vs-humans/test/horse1.jpeg'
img = image.load_img(path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print(path.split('/')[-1] + " is a human")
else:
    print(path.split('/')[-1] + " is a horse")


[0.]
horse1.jpeg is a horse


In [21]:
cuda.close()